In [1]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

In [2]:
class DataAnalysis:
    def __init__(self, num_trials=50):
        self.num_trials = num_trials
    
    def import_data(self, Spath, Upath):
        Sload = np.loadtxt(Spath, delimiter=',')
        Uload = np.loadtxt(Upath, delimiter=',')
        return Sload, Uload
    
    def generate_testvals(self, trial_version):
        RSS_mpc = []
        RSS_mhe = []
        SSU_mpc = []
        SSU_mhe = []
        num_failures = 0.0

        print(f"{"="*25} Generating Test Values {"="*25}")
        for trial in range(self.num_trials):
            try:
                Sref, Uref = self.import_data(f"trajectories/states/dircol_soln_{trial}.csv",
                                              f"trajectories/inputs/dircol_inpu_{trial}.csv")
                
                Smpc, Umpc = self.import_data(f"solves_MPC/states/{trial_version}/mpc_soln_{trial}.csv",
                                              f"solves_MPC/inputs/{trial_version}/mpc_inpu_{trial}.csv")
                
                Smhe, Umhe = self.import_data(f"solves_MHEMPC/states/{trial_version}/mhempc_soln_{trial}.csv",
                                              f"solves_MHEMPC/inputs/{trial_version}/mhempc_inpu_{trial}.csv")
                RSS_mpc.append(np.sum(np.square(Sref - Smpc)))
                RSS_mhe.append(np.sum(np.square(Sref - Smhe)))

                SSU_mpc.append(np.sum(np.square(Umpc)))
                SSU_mhe.append(np.sum(np.square(Umhe)))
                
            except Exception as e:
                print(f"Trial {trial} missing data from one or more controllers")
                num_failures += 1

        print(f"Failure Percentage: {num_failures/self.num_trials * 100}%")
        return np.array(RSS_mpc), np.array(RSS_mhe), np.array(SSU_mpc), np.array(SSU_mhe)
    
    def test_normality(self, a, tol=0.05):
        print(f"{"="*25} Testing Normality {"="*25}")
        s_stat, p_val = stats.shapiro(a)

        print(f"s = {s_stat:.5E} | p = {p_val:.5E}")

        if(p_val < tol):
            print(f"p_val = {p_val} < {tol}, the underlying distribution is likely normal")
            return True
        else: 
            print(f"p_val = {p_val} > {tol}, the underlying distribution is likely not normal")
            return False

    def t_test(self, a, b):
        paired_diff = a - b
        if self.test_normality(paired_diff):
            print(f"{"="*25} Performing t-test {"="*25}")
            t_stat, p_val = stats.ttest_rel(a, b)
            print(f"t = {t_stat:.5E} | p = {p_val:.5E}")
            return t_stat, p_val
        else:
            return -1, -1

In [3]:
num_trials = 50
trial_versions = ["base", "base_vel", "mismatch", "mismatch_vel"]
analyzer = DataAnalysis()

### RSS means t-test
Hypothesis: In an marine environment with external disturbances and model mismatch, a USV controlled by a MPC using a MHE to estimate the dynamics of the system will follow a trajectory with less error than a MPC using a dynamics model derived from calm water scale testing.

H0: There is no difference in the means of the RSS between the MHEMPC and the MPC (ie. MHEMPC and MPC perform similarly)

HA: There is a difference in the means of the RSS between the MHEMPC and the MPC (ie. MHEMPC and MPC perform differently)

In [4]:
for version in trial_versions:
    print(f"Data analysis for: {version}")
    RSS_mpc, RSS_mhe, _, _ = analyzer.generate_testvals(version)
    t_stat, p_val = analyzer.t_test(RSS_mhe, RSS_mpc)

    print(f"{"="*25} RSS Means {"="*25}")
    RSS_mean_mpc = np.mean(RSS_mpc)
    RSS_mean_mhe = np.mean(RSS_mhe)
    print(f"MPC RSS Mean: {RSS_mean_mpc:.5E} | MHEMPC RSS Mean: {RSS_mean_mhe:.5E}")

    print("+"*100)

Data analysis for: base
========================= Generating Test Values =========================
Trial 13 missing data from one or more controllers
Trial 14 missing data from one or more controllers
Trial 40 missing data from one or more controllers
Trial 47 missing data from one or more controllers
Failure Percentage: 8.0%
========================= Testing Normality =========================
s = 4.51274E-01 | p = 6.92573E-12
p_val = 6.925729838335982e-12 < 0.05, the underlying distribution is likely normal
========================= Performing t-test =========================
t = 2.84707E+00 | p = 6.62520E-03
========================= RSS Means =========================
MPC RSS Mean: 3.22531E-01 | MHEMPC RSS Mean: 1.20420E+01
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Data analysis for: base_vel
========================= Generating Test Values =========================
Trial 41 missing data from one or more controllers
Failure

### Average Actuator Effort
Hypothesis: In an marine environment with external disturbances and model mismatch, a USV controlled by a MPC using a MHE to estimate the dynamics of the system will expend less actuator effort to follow a trajectory than a MPC using a dynamics model derived from calm water scale testing.

In [5]:
for version in trial_versions:
    print(f"Data analysis for: {version}")
    _, _, SSU_mpc, SSU_mhe = analyzer.generate_testvals(version)
    t_stat, p_val = analyzer.t_test(RSS_mhe, RSS_mpc)

    print(f"{"="*25} Average Actuator Effort {"="*25}")
    SSU_mean_mpc = np.mean(SSU_mpc)
    SSU_mean_mhe = np.mean(SSU_mhe)
    print(f"MPC SSU Mean: {SSU_mean_mpc:.5E} | MHEMPC SSU Mean: {SSU_mean_mhe:.5E}")

    print("+"*100)

Data analysis for: base
========================= Generating Test Values =========================
Trial 13 missing data from one or more controllers
Trial 14 missing data from one or more controllers
Trial 40 missing data from one or more controllers
Trial 47 missing data from one or more controllers
Failure Percentage: 8.0%
========================= Testing Normality =========================
s = 9.27085E-01 | p = 1.15715E-02
p_val = 0.011571518801414054 < 0.05, the underlying distribution is likely normal
========================= Performing t-test =========================
t = 4.90863E+00 | p = 1.58350E-05
========================= Average Actuator Effort =========================
MPC SSU Mean: 3.17221E+03 | MHEMPC SSU Mean: 4.99124E+03
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Data analysis for: base_vel
========================= Generating Test Values =========================
Trial 41 missing data from one or more contro